# VTOL Example



In [ ]:
import numpy as np
import cvxpy as cp

class VTOL:
    def __init__(self, mass, inertia, arm_len, drag=0.1):
        self.M = mass
        self.J = inertia
        self.mu = drag
        self.d = arm_len
        self.g = 9.81
        self.x = np.zeros((6,1)) # z,h,theta,zd,hd,thetad
        self.dt = 0.01
    
    def updateDynamics(self, u): # RK4 integration
        k1 = self._derivatives(self.x, u)
        k2 = self._derivatives(self.x + k1*self.dt*0.5, u)
        k3 = self._derivatives(self.x + k2*self.dt*0.5, u)
        k4 = self._derivatives(self.x + k3*self.dt, u)
        self.x += (k1 + 2*k2 + 2*k3 + k4) * (self.dt/6.0)
    
    def _derivatives(self, x, u):
        theta = x.item(2)
        zd = x.item(3)
        hd = x.item(4)
        thetad = x.item(5)
        Fr = u.item(0)
        Fl = u.item(1)
        
        # mixer
        F = Fl+Fr
        Tau = self.d*(Fr-Fl)
        
        # calculate x_dot
        z_dot      = zd
        h_dot      = hd
        theta_dot  = thetad
        z_ddot     = -(self.mu*zd+F*np.sin(theta))/self.M
        h_ddot     = F*np.cos(theta)/self.M - self.g
        theta_ddot = Tau/self.J
        
        return np.array([z_dot,h_dot,theta_dot,
                         z_ddot,h_ddot,theta_ddot]).reshape((6,1))
        

m_prop = 0.25
m_body = 1
J_body = 0.0042
d = 0.3
mu = 0.1
g = 9.81

drone = VTOL(m_body+2*m_prop, J_body+2*m_prop*d**2, d, mu)